In [1]:
import sys

import pandas as pd
import bokeh
from bokeh.plotting import figure
from bokeh.io import output_file, output_notebook, show
from bokeh.models import DatetimeTickFormatter, BoxSelectTool, HoverTool, Label, LabelSet, LinearAxis, Range1d
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown

print('Python v' + sys.version)
print('Pandas v' + pd.__version__)
print('Bokeh v' + bokeh.__version__)

Python v3.9.5 (tags/v3.9.5:0a7dcbd, May  3 2021, 17:27:52) [MSC v.1928 64 bit (AMD64)]
Pandas v1.1.4
Bokeh v2.2.3


##### Колоне за све мераче протока

In [2]:
# priprema tipova kolona

dtypes = {'vreme': 'str',
          'I-1-LJB':'float',
          'I-BBV-2':'float',
          'BBV_II-8':'float',
          'I-2-DIV':'float',
          'I-3-MOK':'float',
          'I-4-KR1':'float',
          'I-4-KR2':'float',
          'I-5-STU':'float',
          'I-6-MLJ':'float',
          'I-7-M10':'float',
          'I-7-M11':'float',
          'I-7-M2':'float',
          'I-V-6':'float',
          'MRB':'float', 
          'III-8':'float',
          'M-10-20-1':'float', 
          'M-10-40-3':'float', 
          'M-10-40-4':'float',
          'M-10-50-5':'float',
          'M-10-60-6':'float',
          'M-10-80-7':'float',
          'M-10-80-8':'float',
          'M-100-120-9':'float',
          'M-10-90-10':'float',
          'M-10-100-11':'float',
          'M-10-20-12':'float',
          'M-20-992-13':'float',
          'M-20-30-14':'float',
          'M-20-100-15':'float', 
          'M-40-50-16':'float',
          'M-70-81-17':'float',
          'M-20-40-18':'float',
          'M-50-60-19':'float',
          'M-90-91-20':'float',
          'M-20-40-21':'float',
          'M-20-30-22':'float',         
          'M-20-50-23':'float',
          'M-20-992-27':'float',
          'M-120-150-28':'float', 
          'M-80-140-29':'float',
          'M-10-90-32':'float',          
          'M-170-992-34':'float',
          'M-190-21-35':'float'
         }

# priprema datuma

parse_dates = ['vreme']


### Читање припремљених CSV датотека за годину мерења:

In [3]:
def q_csv(godina):
    
    # čitanje kreiranog CSV fajla sa merenim protocima (SCADA_2021.csv)
    # df_q = pd.read_csv("SCADA_2021.csv", sep=',', header=0, dtype=dtypes, parse_dates=parse_dates, infer_datetime_format=True)

    csv_ime = "SCADA_" + godina + ".csv"
    q = pd.read_csv(csv_ime, sep=',', header=0, dtype=dtypes, parse_dates=parse_dates, infer_datetime_format=True)

    # određivanje indeks kolone
    q.set_index("vreme", inplace=True)
    
    return q

def p_csv(godina):
    
    # čitanje kreiranog CSV fajla sa merenim pritiscima (SCADA_2021_p.csv)
    # df_p = pd.read_csv("SCADA_2021_p.csv", sep=',', header=0, dtype=dtypes, parse_dates=parse_dates, infer_datetime_format=True)
   
    csv_ime = "SCADA_" + godina + "_p.csv"
    p = pd.read_csv(csv_ime, sep=',', header=0, dtype=dtypes, parse_dates=parse_dates, infer_datetime_format=True)
    
    # određivanje indeks kolone
    p.set_index("vreme", inplace=True)
    
    return p

def rez_csv(godina):
    
    # čitanje kreiranog CSV fajla sa merenim nivoia rezervoara (SCADA_2021_REZ.csv)
    # df_p = pd.read_csv("SCADA_2021_REZ.csv", sep=',', header=0, dtype=dtypes, parse_dates=parse_dates, infer_datetime_format=True)
   
    csv_ime = "SCADA_" + godina + "_REZ.csv"
    rez = pd.read_csv(csv_ime, sep=',', header=0, dtype=dtypes, parse_dates=parse_dates, infer_datetime_format=True)
    
    # određivanje indeks kolone
    rez.set_index("vreme", inplace=True)
    
    return rez


In [4]:
print(q_csv("2021").columns)

print(rez_csv("2021").columns)

Index(['I-1-LJB', 'I-BBV-2', 'BBV_II-8', 'I-2-DIV', 'I-3-MOK', 'I-4-KR1',
       'I-4-KR2', 'I-V-6', 'I-5-STU', 'MRB', 'III-8', 'M-70-81-17', 'I-7-M10',
       'I-7-M11', 'I-7-M2', 'I-6-MLJ', 'M-20-992-13', 'M-20-992-27',
       'M-170-992-34', 'M-190-21-35', 'M-100-120-9', 'M-10-100-11',
       'M-10-20-1', 'M-10-20-12', 'M-10-40-3', 'M-10-40-4', 'M-10-50-5',
       'M-10-60-6', 'M-10-80-7', 'M-10-80-8', 'M-10-90-10', 'M-10-90-32',
       'M-120-150-28', 'M-20-100-15', 'M-20-30-14', 'M-20-30-22', 'M-20-40-18',
       'M-20-40-21', 'M-20-50-23', 'M-40-50-16', 'M-50-60-19', 'M-80-140-29',
       'M-90-91-20'],
      dtype='object')
Index(['Vinik', 'Krupac', 'Bubanj', 'Matejevac_2', 'Miljkovac', 'Toplik_B',
       'Toplik_PS'],
      dtype='object')


In [5]:
# df_q = q_csv("2020")
# df_p = p_csv("2020")

# df_q = q_csv("2021")
# df_p = p_csv("2021")
# df_rez = rez_csv("2021")

# df_q
# df_p

### Читање логова за поједине мераче:

In [6]:
def mp_log(df, db_mp):
    
    return df[db_mp]   

#### Протоци ( Q)

In [7]:
# definisanje kolona

vreme = q_csv("2021").index
LJB = mp_log(q_csv("2021"),'I-1-LJB')          # 1
BNI = mp_log(q_csv("2021"),'I-BBV-2')          # 2
BAB = mp_log(q_csv("2021"),'BBV_II-8')         # 3
DIV = mp_log(q_csv("2021"),'I-2-DIV')          # 4
MOK = mp_log(q_csv("2021"),'I-3-MOK')          # 5
KR1 = mp_log(q_csv("2021"),'I-4-KR1')          # 6
KR2 = mp_log(q_csv("2021"),'I-4-KR2')          # 7
VIN = mp_log(q_csv("2021"),'I-V-6')            # 8
VN1 = mp_log(q_csv("2021"),'M-20-992-13')      # 9
VN2 = mp_log(q_csv("2021"),'M-20-992-27')      # 10
VN3 = mp_log(q_csv("2021"),'M-170-992-34')     # 11
VN4 = mp_log(q_csv("2021"),'M-190-21-35')      # 12

M10 = mp_log(q_csv("2021"),'I-7-M10')          # 13
M11 = mp_log(q_csv("2021"),'I-7-M11')          # 14
M20 = mp_log(q_csv("2021"),'I-7-M2')           # 15

STU = mp_log(q_csv("2021"),'I-5-STU')          # 16
MRB = mp_log(q_csv("2021"),'MRB')              # 17

MET = mp_log(q_csv("2021"),'III-8')            # 18
BUB = mp_log(q_csv("2021"),'M-70-81-17')       # 19

MLJ = mp_log(q_csv("2021"),'I-6-MLJ')          # 20

KNJ = mp_log(q_csv("2021"),'M-20-30-14')       # 21
DJM = mp_log(q_csv("2021"),'M-20-30-22')       # 22

NTX = mp_log(q_csv("2021"),'M-10-20-1')        # 23
ROD = mp_log(q_csv("2021"),'M-10-40-3')        # 24

In [8]:
# print(vreme)
# print(LJB)
# print(LJB.index)
# print(LJB[0])
# print(LJB.index[0], LJB[0])

#### Притисци (p)

In [9]:
# definisanje kolona
vreme_p = p_csv("2021").index
LJB_p = mp_log(p_csv("2021"),'I-1-LJB')          # 1
BNI_p = mp_log(p_csv("2021"),'I-BBV-2')          # 2
BAB_p = mp_log(p_csv("2021"),'BBV_II-8')         # 3
DIV_p = mp_log(p_csv("2021"),'I-2-DIV')          # 4
MOK_p = mp_log(p_csv("2021"),'I-3-MOK')          # 5
KR1_p = mp_log(p_csv("2021"),'I-4-KR1')          # 6
KR2_p = mp_log(p_csv("2021"),'I-4-KR2')          # 7
VIN_p = mp_log(p_csv("2021"),'I-V-6')            # 8
VN1_p = mp_log(p_csv("2021"),'M-20-992-13')      # 9
VN2_p = mp_log(p_csv("2021"),'M-20-992-27')      # 10
VN3_p = mp_log(p_csv("2021"),'M-170-992-34')     # 11
VN4_p = mp_log(p_csv("2021"),'M-190-21-35')      # 12
   
M10_p = mp_log(p_csv("2021"),'I-7-M10')          # 13
M11_p = mp_log(p_csv("2021"),'I-7-M11')          # 14
M20_p = mp_log(p_csv("2021"),'I-7-M2')           # 15

STU_p = mp_log(p_csv("2021"),'I-5-STU')          # 16
MRB_p = mp_log(p_csv("2021"),'MRB')              # 17
MET_p = mp_log(p_csv("2021"),'III-8')            # 18
BUB_p = mp_log(p_csv("2021"),'M-70-81-17')       # 19

MLJ_p = mp_log(p_csv("2021"),'I-6-MLJ')          # 20

KNJ_p = mp_log(p_csv("2021"),'M-20-30-14')       # 21
DJM_p = mp_log(p_csv("2021"),'M-20-30-22')       # 22

NTX_p = mp_log(p_csv("2021"),'M-10-20-1')        # 23
ROD_p = mp_log(p_csv("2021"),'M-10-40-3')        # 24


#### Резервоари (ниво у метрима)

In [10]:
# definisanje kolona
vreme_rez = rez_csv("2021").index

VIN_r = mp_log(rez_csv("2021"),'Vinik')           # 1
KRP_r = mp_log(rez_csv("2021"),'Krupac')          # 2
BUB_r = mp_log(rez_csv("2021"),'Bubanj')          # 3
MAT_r = mp_log(rez_csv("2021"),'Matejevac_2')     # 4
MLJ_r = mp_log(rez_csv("2021"),'Miljkovac')       # 5
TPB_r = mp_log(rez_csv("2021"),'Toplik_B')        # 6
TPS_r = mp_log(rez_csv("2021"),'Toplik_PS')       # 7


## подешавања дијаграма (bokeh)

In [11]:
TOOLS = ["xpan", "xwheel_zoom", "box_zoom", "reset", "save"]

# TOOLTIPS = [
#     ("MP", "$name"),
#     ("(Q,t)", "($y, $x)")
# ]


hover = HoverTool(tooltips=[
    ("MP", "$name"),
    ("Q", "$y{0.0 a}"),
    ("datum", "$x{%F}"),
    ("vreme", "$x{%H:%M}")
#   ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
#   ( 'volume', '@volume{0.00 a}'      ),
    ],

    formatters={
        "$x": 'datetime'      #, use 'datetime' formatter for '@date' field
#       '@{adj close}':'printf', # use 'printf' formatter for '@{adj close}' field
                                 # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='mouse'
)

In [12]:
# postavke za bokeh dijagram protoka

f = figure(
           plot_width=1500,
           plot_height=500,
           x_axis_type='datetime',
           y_range=(-200, 1400),
           tools=TOOLS+[hover],
#            tooltips=TOOLTIPS,
           active_scroll='xwheel_zoom',
           background_fill_color="#f9f9f9",
           toolbar_location="above"
          )     

f.add_tools(BoxSelectTool(dimensions="width"))

f.title.text_color="blue"
f.title.text_font="arial"
f.title.text_font_style="bold"
f.title.text_font_size="14pt" # "1.5em" 
f.title.text_alpha=1 #floating point between 0 (transparent) and 1 (opaque)

f.xaxis.minor_tick_line_color=None
f.yaxis.minor_tick_line_color=None
f.xaxis.axis_label="Време"
f.yaxis.axis_label="Проток (лит/сек)" 

f.xaxis.formatter=DatetimeTickFormatter(hours=["%H:%M"],days=["%d %B"],months=["%B %Y"],years=["%Y"])

f.toolbar.autohide = True

In [13]:
# postavke za bokeh dijagram pritisaka

p = figure(
           plot_width=1500,
           plot_height=14,
           x_axis_type='datetime',
           y_range=(-5, 13),
           tools=TOOLS+[hover],
#            tooltips=TOOLTIPS,
           active_scroll='xwheel_zoom',
           background_fill_color="#f9f9f9",
           toolbar_location="above"
          )     

p.add_tools(BoxSelectTool(dimensions="width"))

p.title.text_color="green"
p.title.text_font="arial"
p.title.text_font_style="bold"
p.title.text_font_size="14pt" # "1.5em" 
p.title.text_alpha=1 #floating point between 0 (transparent) and 1 (opaque)

p.xaxis.minor_tick_line_color=None
p.yaxis.minor_tick_line_color=None
p.xaxis.axis_label="Време"
p.yaxis.axis_label="Притисак (бар)" 

p.xaxis.formatter=DatetimeTickFormatter(hours=["%H:%M"],days=["%d %B"],months=["%B %Y"],years=["%Y"])

p.toolbar.autohide = True

In [14]:
# postavke za bokeh dijagram nivoa u rezervoarima

r = figure(
           plot_width=1500,
           plot_height=14,
           x_axis_type='datetime',
           y_range=(-5, 13),
           tools=TOOLS+[hover],
#            tooltips=TOOLTIPS,
           active_scroll='xwheel_zoom',
           background_fill_color="#f9f9f9",
           toolbar_location="above"
          )     

r.add_tools(BoxSelectTool(dimensions="width"))

r.title.text_color="cyan"
r.title.text_font="arial"
r.title.text_font_style="bold"
r.title.text_font_size="14pt" # "1.5em" 
r.title.text_alpha=1 #floating point between 0 (transparent) and 1 (opaque)

r.xaxis.minor_tick_line_color=None
r.axis.minor_tick_line_color=None
r.xaxis.axis_label="Време"
r.yaxis.axis_label="Ниво (метара)" 

r.xaxis.formatter=DatetimeTickFormatter(hours=["%H:%M"],days=["%d %B"],months=["%B %Y"],years=["%Y"])

r.toolbar.autohide = True

#### ИЗВОРИШТА (појединачне криве протока)

In [15]:
def izvorista_mp():
    
    f.title.text="НИВОС - Испоручена вода на извориштима 2021"

    f.line(vreme, LJB, color="brown", name="Љуберађа", legend_label = 'Љуберађа')
    f.line(vreme, DIV, color="red", name="Дивљана", legend_label = 'Дивљана')
    f.line(vreme, MOK, color="purple", name="Мокра", legend_label = 'Мокра')
    f.line(vreme, KR1, color="orange", name="Крупац (стара ПС)", legend_label = 'Крупац (стара ПС)')
    f.line(vreme, KR2, color="yellow", name="Крупац (нова ПС)", legend_label = 'Крупац (нова ПС)')
    f.line(vreme, STU, color="cyan", name="Студена", legend_label = 'Студена')
    f.line(vreme, MLJ, color="green", name="Миљковац", legend_label = 'Миљковац')
    f.line(vreme, M10, color="blue", name="Медијана 1", legend_label = 'Медијана 1')
    f.line(vreme, M11, color="skyblue", name="Медијана (потисни)", legend_label = 'Медијана (потисни)')
    f.line(vreme, M20, color="powderblue", name="Медијана 2", legend_label = 'Медијана 2')

    f.legend.location = 'top_left'
    f.legend.title = 'Мерач протока'
    f.legend.title_text_font = 'Arial'
    f.legend.title_text_font_size = '10pt'
    f.legend.items.reverse()

    return show(f)

#### ИЗВОРИШТА (кумулативна крива протока)

In [16]:
def izvorista_mp_kumulativ():

    f.title.text="НИВОС - Испоручена вода на извориштима 2021 - кумулативно"

#   SVA izvorišta
    mm = ['I-1-LJB', 'I-2-DIV', 'I-3-MOK','I-4-KR1','I-4-KR2','I-5-STU','I-6-MLJ','I-7-M10','I-7-M11','I-7-M2']
    nazivi = ['Љуберађа', 'Дивљана', 'Мокра', 'Крупац (стара ПС)', 'Крупац (нова ПС)', 'Студена', 'Миљковац', 'Медијана 1', 'Медијана (потисни)', 'Медијана 2']
    boje = ['brown', 'red', 'purple', 'orange', 'yellow', 'cyan', 'green', 'blue', 'skyblue', 'powderblue']

#     Bez Studene
#     mm = ['I-1-LJB', 'I-2-DIV', 'I-3-MOK','I-4-KR1','I-4-KR2','I-6-MLJ','I-7-M10','I-7-M11','I-7-M2']
#     nazivi = ['Љуберађа', 'Дивљана', 'Мокра', 'Крупац (стара ПС)', 'Крупац (нова ПС)', 'Миљковац', 'Медијана 1', 'Медијана (потисни)', 'Медијана 2']
#     boje = ['brown', 'red', 'purple', 'orange', 'yellow', 'green', 'blue', 'skyblue', 'powderblue']

    f.varea_stack(stackers=mm, x='vreme', color=boje, legend_label=nazivi, source=df_q)

    f.legend.location = 'top_left'
    f.legend.title = 'Мерач протока'
    f.legend.title_text_font = 'Arial'
    f.legend.title_text_font_size = '10pt'
    f.legend.items.reverse()

    return show(f)

#### Љуберађа - Виник (Изворишта на магистралном цевоводу)

In [17]:
def ljuberadja_vinik():
    
    mm = ['I-1-LJB', 'I-2-DIV', 'I-3-MOK','I-4-KR1','I-4-KR2']
    nazivi = ['Љуберађа', 'Дивљана', 'Мокра', 'Крупац (стара ПС)', 'Крупац (нова ПС)']
    boje = ['brown', 'red', 'purple', 'orange', 'yellow']

    f.varea_stack(stackers=mm, x='vreme', color=boje, legend_label=nazivi, source=df_q)
    f.line(vreme, VIN, color="blue", line_width = 2, name="Виник - улаз", legend_label = 'Виник - улаз')

    f.legend.location = 'top_left'
    f.legend.title = 'Мерач протока'
    f.legend.title_text_font = 'Arial'
    f.legend.title_text_font_size = '10pt'
    f.legend.items.reverse()

    return show(f)

In [18]:
def vinik():
    
    # РЕЗЕРВОАР ВИНИК

    f.line(vreme, VIN, color="blue", line_width = 2, name="Виник - улаз", legend_label = 'Виник - улаз')
    f.line(vreme, VN1, color="green", name="Виник - према граду", legend_label = 'Виник - према граду')
    f.line(vreme, VN2, color="lightseagreen", name="Виник - према Комрену", legend_label = 'Виник - према Комрену')
    f.line(vreme, VN3, color="cyan", name="ПС Виник Довод", legend_label = 'ПС Виник Довод')
    f.line(vreme, VN4, color="lightsteelblue", name="ПС Виник Одвод", legend_label = 'ПС Виник Одвод')

    f.line(vreme, LJB, color="brown", name="Љуберађа", legend_label = 'Љуберађа')
    f.line(vreme, DIV, color="red", name="Дивљана", legend_label = 'Дивљана')
    f.line(vreme, MOK, color="purple", name="Мокра", legend_label = 'Мокра')

    f.line(vreme, BNI, color="red", name="ПС Бабушница -> Ниш", legend_label = 'ПС Бабушница -> Ниш')
    f.line(vreme, BAB, color="purple", name="рез. Бабушница", legend_label = 'рез. Бабушница')

    f.legend.location = 'top_left'
    f.legend.title = 'Мерач протока'
    f.legend.title_text_font = 'Arial'
    f.legend.title_text_font_size = '10pt'
    f.legend.items.reverse()

    return show(f)

### МЕРНЕ ЗОНЕ

#### Мерна Зона 30

In [19]:
def mz_30():
    
    f.title.text="Мерна Зона 30"

    f.line(vreme, M20, color="powderblue", name="Медијана 2")
    f.line(vreme, DJM, color="red", name="Ђачко острво (Медијана)")
    f.line(vreme, KNJ, color="green", name="Књажевачка")
    
    return show(f)

### Мерачи протока - појединачно

#### МРБ

In [20]:
def mp_MRB():
    
    f.title.text="МРБ"
    
    f.line(vreme, MRB, color="green", name="МРБ проток")

    return show(f)

## СИТУАЦИЈЕ

#### Хаварија "Делта" (АЦ 500) 23-октобар-2020

In [21]:
def delta():
    
    vreme = q_csv("2020").index

    VN1 = mp_log(q_csv("2020"),'M-20-992-13')
    NTX = mp_log(q_csv("2020"),'M-10-20-1')
    ROD = mp_log(q_csv("2020"),'M-10-40-3')

    labela =Label(x=70, y=70, x_units='screen', text='LABELA', render_mode='css', 
                  border_line_color='black', border_line_alpha=1.0,
                  background_fill_color='white', background_fill_alpha=1.0)
    f.add_layout(labela)

    f.title.text="Оштећење цевовода АЦ ∅500 - 23. Октобар 2020"

    f.line(vreme, VN1, line_width = 2, color="powderblue", name="Виник 1 (подвожњак)", legend_label = 'Виник 1 (подвожњак)')
    f.line(vreme, NTX, line_width = 1, color="red", name="Нитекс", legend_label = 'Нитекс')
    f.line(vreme, ROD, line_width = 1, color="green", name="Рода (Меркатор)", legend_label = 'Рода (Меркатор)')

    f.legend.location = 'top_left'
    f.legend.title = 'Мерач протока'
    f.legend.title_text_font = 'Arial'
    f.legend.title_text_font_size = '10pt'
    
    return show(f)

#### II Висинска зона (југ) - Јун 2021 (Паси Пољана криза )

In [22]:
def pasi_poljana():
    
    f = figure(
           plot_width=1500,
           plot_height=500,
           x_axis_type='datetime',
           y_range=(0, 150),
           tools=TOOLS+[hover],
#            tooltips=TOOLTIPS,
           active_scroll='xwheel_zoom',
           background_fill_color="#f9f9f9",
           toolbar_location="above"
          )   
    
    f.xaxis.axis_label = 'време'
    f.yaxis.axis_label = 'lit/sec'
    f.xaxis.formatter=DatetimeTickFormatter(hours=["%H:%M"],days=["%d %B"],months=["%B %Y"],years=["%Y"])
    
    
    f.title.text="II Висинска зона (југ) 2021 "

    f.line(vreme, MET, line_width = 2, color="powderblue", name="Метох", legend_label = 'Метох [III-8]')
    f.line(vreme, BUB, line_width = 1, color="red", name="Бубањ", legend_label = 'Бубањ [M-70-81-17]')
        
    f.extra_y_ranges = {"Rez": Range1d(start = 0, end = 6)}
    f.add_layout(LinearAxis(y_range_name = "Rez", axis_label="Рез. ниво [m]"), 'right')
    f.line(vreme, BUB_r, line_width = 2, color="blue", name="рез. Бубањ", legend_label = 'рез. Бубањ [m] ', y_range_name = "Rez")

    f.legend.location = 'top_left'
    f.legend.title = 'Мерач'
    f.legend.title_text_font = 'Arial'
    f.legend.title_text_font_size = '10pt'
                   
    return show(f)
    

# Криве мерења

In [23]:
# izvorista_mp()

# izvorista_mp_kumulativ()

# ljuberadja_vinik()

# vinik()

# MZ_30()

# mp_MRB()

# delta()

pasi_poljana()

In [24]:
# menu = [("Option 1", "item_1"), ("Option 2", "item_2")]
# dropdown = Dropdown(label="Dropdown Menu", button_type="warning", menu=menu)

# f.legend.location = 'top_left'
# f.legend.title = 'Мерачи протока 2021'
# f.legend.title_text_font = 'Arial'
# f.legend.title_text_font_size = '10pt'



In [25]:
# from bokeh.models import ColumnDataSource
# from bokeh.plotting import figure, show

# data = {
#     "months": [date(2019, 5, 1), date(2019, 6, 1), date(2019, 7, 1), date(2019, 8, 1)],
#     "dogs": [7, 5, 12, 8],
#     "cats": [25, 20, 10, 7],
# }

# column_data = ColumnDataSource(data=data)

# p = figure(
#     min_width=500,
#     plot_height=300,
#     sizing_mode="stretch_width",
#     toolbar_location=None,
#     tools="",
#     x_axis_type="datetime",
# )

# p.varea_stack(
#     ["dogs", "cats"],
#     x="months",
#     color=["#ff0000", "#00ff00"],
#     source=column_data,
# )

# show(p)

In [26]:
# import numpy as np
# import pandas as pd

# from bokeh.palettes import brewer
# from bokeh.plotting import figure, output_file, show

# output_file('stacked_area.html')

# N = 3
# df = pd.DataFrame(np.random.randint(10, 100, size=(15, N))).add_prefix('y')

# p = figure(x_range=(0, len(df)-1), y_range=(0, 800))
# p.grid.minor_grid_line_color = '#eeeeee'

# names = ["y%d" % i for i in range(N)]
# p.varea_stack(stackers=names, x='index', color=brewer['Spectral'][N], legend_label=names, source=df)

# # reverse the legend entries to match the stacked order
# p.legend.items.reverse()

# show(p)